In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from bayes_opt import BayesianOptimization

from sklearn.neighbors import KDTree
import hdbscan
# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2, w1, w2, w3, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
    
#     if m == 1:
#         print(da)
    
    z1 = z/r
    z2 = z/d
    
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
#     X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
    cx = [w1,w1,w2,w3]
    for jj in range(1,X.shape[1]): 
         X[:,jj] = X[:,jj]*cx[jj]
#     l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
    _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)
#     a = np.arctan2(z, r)
    
    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r
    
    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))

    theta= np.arctan2(r, z)
    theta1 = np.arctan2(r2, z2)

    
    tan_dip = phi/theta
    tan_dip1 = phi/z2
    z2_1 = 1/z2
    z2_2 = phi/z2 + 1/z2
    
    dip_angle = np.arctan2(z2, (np.sqrt(x2**2 +y2**2)) * np.arccos(x2/np.sqrt(x2**2 + y2**2)))
    dip_angle1 = np.arctan2(z, (np.sqrt(x**2 +y**2)) * np.arccos(x2/np.sqrt(x**2 + y**2)))
    scores = []

    a_start,a_step,a_num = 0.00100,0.0000095,150
    
    
    a_num = Niter.astype(np.int32)
    params  = [(i,m, x,y,z,d,r,r2,z2, w1, w2, w3, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

    if 1: 
        pool = Pool(processes=1)
        ls   = pool.map( one_loop1, params )


    if 0:
        ls = [ one_loop(param) for param in  params ]


    ##------------------------------------------------

    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
       
    
#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
def Fun4BO(w1,w2,w3,Niter):
    track_id = do_dbscan_predict(hits, w1,w2,w3,Niter)
#     track_id = do_dbscan_predict2(hits)
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(2):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [7]:

#########################################

# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')

#     hits = hits[hits.particle_id != 0]
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
 179.0000000[ 0] score : 0.51318437
 179.0000000[ 1] score : 0.51721947
--------------------------------------
0.5152019205697601
    1 | 05m23s |    0.51520 |  154.5319 |    1.1820 |    0.6159 |    0.1013 | 
 179.0000000[ 0] score : 0.51814982
 179.0000000[ 1] score : 0.52142011
--------------------------------------
0.5197849648342838
    2 | 05m02s |    0.51978 |  145.5130 |    0.9940 |    0.5367 |    0.1956 | 
 179.0000000[ 0] score : 0.51799722
 179.0000000[ 1] score : 0.52263257
--------------------------------------
0.5203148944415414
    3 | 06m02s |    0.52031 |  177.2357 |    1.0440 |    0.5970 |    0.3689 | 
Bayesian Optimization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
 179.0000000[ 0] score

In [6]:
#########################################

# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')

#     hits = hits[hits.particle_id != 0]
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    
    track_id = do_dbscan_predict(hits)

    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

print('--------------------------------------')
print(sum_score/sum)


 179.0000000[ 0] score : 0.49409776
 179.0000000[ 1] score : 0.49873591
 179.0000000[ 2] score : 0.50057185
 179.0000000[ 3] score : 0.50147617
 179.0000000[ 4] score : 0.50191906
 179.0000000[ 5] score : 0.50177841
 179.0000000[ 6] score : 0.50144026
 179.0000000[ 7] score : 0.50175792
--------------------------------------
0.5002221667884463


In [7]:
# run_dbscan()
# 0.38886407617656094
# 0.43999828380439265 after caa, saa
# 0.48344805750948866 after extend
# 0.4835126378397637 after removing aa/zr + 1/zr
# with z2 goes down to 0.45
# with r2 goes down to 0.40
# with 1/r2 down to 0.43
# with 1/r2 and theta down to 0.41
# with 1/r2 and 1/theta down to 0.41
# 0.13 with phi2
# 0.417 with 1/phi2
# 0.436 with tan_dip1
# 0.422 with 1/tan_dip1
# 0.434 with z2r2 and z2r2 and 1/z2r2
# 0.334 if zr is replaced with z2r2
# 0.4888877339626617 if aa/zr replaced with caa/zr
# 0.4887566511321033 if aa/zr if replaces with saa/zr
# 0.4895407603454436 if aa/zr is removed ( only caa, saa, zr, 1/zr remains)
# 0.48876599546214305 with 0.0038 instead of 0.0035
# 0.48769542639758245 if remove 1/zr
# 0.0 if both zr and 1/zr are removed
# 0.44026470041552757 if ct, st are added
# 0.21311494225786964 if only caa, saa, ct
# 0.40320264914764326 if only caa, saa, st
# 0.3773604786775556 if only caa, saa, st, 1/st
# 0.4895407603454436 if only caa, saa, tt, 1/tt
# 0.13480299643426075 if only taa, 1/taa, tt, 1/tt
# 0.0885347888971582 if only taa, 1/taa, ct, st
# 0.04670868936897901 if only taa, ct, st
# 0.025523671048730837 if only taa and st
# 0.025057657406745617 if only 1/taa, st
# 0.5025429199508328 caa, saa, tt, 1/tt  when particle_id != 0
# 0.4909138125112928 with deps=0.000005
# 0.4884101555506272 with deps=0.000010
# 0.49112882444220873 with deps=0.0000025
# 0.4903305156288079 with deps=0.0000021
# 0.4900532530976269 with deps=0.0000022
# 0.4908567128710496 with deps=0.0000023
# 0.4906372351101146 with deps=0.0000024
# 0.49066011553021205 with deps=0.0000026
# 0.4907583816106307 with deps=0.0000027
# 0.49015657761806697 with deps=0.0000028
# 0.49027772717858303 with deps=0.0000029
# 0.48968636878606386 wih deps=0.00000025
# 0.48868485370179743 with 180 steps
# 0.4922564475970125 with 170 steps
# 0.4931891169309856 with 160 steps
# 0.4940977643648564 with 150 steps
# 0.4905422457697099 with 140 steps
# 0.49255846616829907 with 130 steps
# 0.4886465357416342 with 120 steps
# 0.3215078003096232 with 110 steps
# 0.49279218495929467 with 155 steps
# 0.4918913043654569 with 145 steps

In [13]:
track_id

NameError: name 'track_id' is not defined

In [35]:
l = pd.Series([1,2,0,4,5,19,34,56,2,4,7777,5, 9822, 19, 0, 2, 1])
print(np.unique(l, return_counts=True, return_inverse=True))
_, reverse, count = np.unique(l, return_counts=True, return_inverse=True)

(array([   0,    1,    2,    4,    5,   19,   34,   56, 7777, 9822]), array([1, 2, 0, 3, 4, 5, 6, 7, 2, 3, 8, 4, 9, 5, 0, 2, 1]), array([2, 2, 3, 2, 2, 2, 1, 1, 1, 1]))


In [36]:
counts = count[reverse]
print(counts)

[2 3 2 2 2 2 1 1 3 2 1 2 1 2 2 3 2]


In [40]:
list(l).count(0)

2

In [26]:
count

array([2, 2, 3, 2, 2, 2, 1, 1, 1, 1])

In [43]:
np.where(np.array(l)==0)

(array([ 2, 14]),)

In [46]:
np.where(pd.Series(l)==0)

(array([ 2, 14]),)

In [50]:
pd.Series(l)==0

0     False
1     False
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14     True
15    False
16    False
dtype: bool

In [86]:
l = pd.Series(l)

In [87]:
l[l == 0] = 10

In [88]:
l

0        1
1        2
2       10
3        4
4        5
5       19
6       34
7       56
8        2
9        4
10    7777
11       5
12    9822
13      19
14      10
15       2
16       1
dtype: int64

In [89]:
list(l)

[1, 2, 10, 4, 5, 19, 34, 56, 2, 4, 7777, 5, 9822, 19, 10, 2, 1]

In [70]:
X = hits[['x2','y2','z2']].values

In [71]:
X.shape

(113998, 3)

In [73]:
l1 = pd.Series(l)

In [83]:
s = list(l1[l1 == 0].keys())

In [84]:
X[s]

array([[-4.29421146e-02,  1.77294928e-03, -2.32458407e+01],
       [-5.05565632e-02,  7.16865790e-05, -1.97545112e+01]])

In [ ]:
num_hits=6
labels = np.zeros(num_hits,np.int32)
counts = np.zeros(num_hits,np.int32)
for l in ls:
    c = make_counts(l)
    print(len(c))
    print(len(l))
    idx = np.where((c-counts>0) & (c<20))[0]
    labels[idx] = l[idx] + labels.max()
    counts = make_counts(labels)
    print(len(counts))
    print('--------------')

In [21]:
num_hits=17
labels = np.zeros(num_hits,np.int32)
counts = np.zeros(num_hits,np.int32)

In [22]:
l

NameError: name 'l' is not defined

In [108]:
c = make_counts(l)

In [109]:
c

array([2, 3, 0, 2, 2, 2, 1, 1, 3, 2, 1, 2, 1, 2, 0, 3, 2])

In [112]:
counts

array([0, 0, 0, 0, 0, 0], dtype=int32)

In [119]:
c-counts > 0

array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True])

In [125]:
np.where((c-counts>0) & (c<2))[0]

array([ 6,  7, 10, 12])

In [126]:
idx = np.where((c-counts>0) & (c<2))[0]

In [127]:
labels[idx] = l[idx] + labels.max()

In [128]:
labels

array([   0,    0,    0,    0,    0,    0,   34,   56,    0,    0, 7777,
          0, 9822,    0,    0,    0,    0], dtype=int32)